In [1]:
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import os
import shutil

# Define the transform to apply to the images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create the directory structure if it doesn't exist
os.makedirs('./data/cifar10/train', exist_ok=True)
os.makedirs('./data/cifar10/test', exist_ok=True)

# Download the training dataset
trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
# Download the testing dataset
testset = CIFAR10(root='./data', train=False, download=True, transform=transform)


# Function to save images to folders
def save_images_to_folder(dataset, folder_path):
    for i, (image, label) in enumerate(dataset):
        class_folder = os.path.join(folder_path, str(label))
        os.makedirs(class_folder, exist_ok=True)
        image_path = os.path.join(class_folder, f"{i}.png")
        torchvision.utils.save_image(image, image_path)

# Save training images
save_images_to_folder(trainset, './data/cifar10/train')
# Save testing images
save_images_to_folder(testset, './data/cifar10/test')

print("CIFAR-10 images saved to train and test folders.")

100%|██████████| 170M/170M [00:08<00:00, 19.4MB/s]


CIFAR-10 images saved to train and test folders.


In [11]:
import tensorflow as tf

# Define the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
import tensorflow as tf

# Define image size and batch size
img_height, img_width = 32, 32
batch_size = 32

# Create image data generators
train_ds = tf.keras.utils.image_dataset_from_directory(
    './data/cifar10/train',
    labels='inferred',
    label_mode='int',
    image_size=(img_height, img_width),
    interpolation='nearest',
    batch_size=batch_size,
    shuffle=True
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    './data/cifar10/test',
    labels='inferred',
    label_mode='int',
    image_size=(img_height, img_width),
    interpolation='nearest',
    batch_size=batch_size,
    shuffle=False
)


Found 50000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.


In [16]:
# Normalize pixel values to be between 0 and 1
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Train the model
model.fit(train_ds, epochs=1)


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 62s 40ms/step - accuracy: 0.5729 - loss: 1.2269


In [17]:
# Evaluate the model
loss, accuracy = model.evaluate(test_ds)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.5964 - loss: 1.1669
Test loss: 1.1955
Test accuracy: 0.5860
